#### This project focuses on unsupervised learning using Pubmed abstracts on Alzheimer's Disease.  
#### 1. Data collection step from PubMed using Biopython.

In [1]:
# connect to aws
from sshtunnel import SSHTunnelForwarder

# SSH config shortcut
SSH_CONFIG_SHORTCUT = 'if applicable'

server = SSHTunnelForwarder(
    SSH_CONFIG_SHORTCUT,
    ssh_config_file='~/.ssh/config',
    remote_bind_address=('localhost', 5432)
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 50580


In [2]:
from pymongo import MongoClient
client = MongoClient(port=12345) # this is the port set by the SSH tunnel
db = client.pubmed_ad # database name

In [40]:
# helper funcs to get data to mongoDB
import time
from Bio import Entrez, Medline
import json
alz = db.alz

def filter_record(record):
    return record

def write_to_mongo(record):
    alz.insert_one(record)

def write_to_file(f, record):
    record = {k: v for k, v in record.items() if k != '_id'}
    f.write(json.dumps(record))
    f.write('\n')

In [24]:
# can you stringify a dict?
test = {'is': 5, 'id': [1,4, 5]}
str(test)

"{'is': 5, 'id': [1, 4, 5]}"

In [59]:
# modified from biopython docs. Errors in documentation! 

from urllib.error import HTTPError  # for Python 3

Entrez.email = "your_email@gmail.com"
search_results = Entrez.read(Entrez.esearch(db="pubmed",
                                            term="alzheimer\'s",
                                            usehistory="y"))
count = int(search_results["Count"])
print("Found %i results" % count)
batch_size = 1000

# two lines below to test
# count = 6
# batch_size = 2


out_handle = open('backup.json', 'a')
for start in range(0,count,batch_size):
    time.sleep(0.3)
    end = min(count, start+batch_size)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 1
    while attempt <= 3:
        try:
            fetch_handle = Entrez.efetch(db="pubmed",rettype="medline",
                                         retmode="text",retstart=start,
                                         retmax=batch_size,
                                         webenv=search_results["WebEnv"],
                                         query_key=search_results["QueryKey"])
            #print('finished fetch')
            attempt = 4 # added this line else enters infinite loop
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                attempt += 1
                time.sleep(15)
            else:
                raise
    records = Medline.parse(fetch_handle) # this and next 4 lines deviate from docs
    for i, record in enumerate(records):
        record = filter_record(record)
        write_to_mongo(record)
        write_to_file(out_handle, record)
out_handle.close()

Found 130672 results
Going to download record 1 to 1000
Going to download record 1001 to 2000
Going to download record 2001 to 3000
Going to download record 3001 to 4000
Going to download record 4001 to 5000
Going to download record 5001 to 6000
Going to download record 6001 to 7000
Going to download record 7001 to 8000
Going to download record 8001 to 9000
Going to download record 9001 to 10000
Going to download record 10001 to 11000
Going to download record 11001 to 12000
Going to download record 12001 to 13000
Going to download record 13001 to 14000
Going to download record 14001 to 15000
Going to download record 15001 to 16000
Going to download record 16001 to 17000
Going to download record 17001 to 18000
Going to download record 18001 to 19000
Going to download record 19001 to 20000
Going to download record 20001 to 21000
Going to download record 21001 to 22000
Going to download record 22001 to 23000
Going to download record 23001 to 24000
Going to download record 24001 to 25000
G

In [ ]:
!wc -l backup.json

In [60]:
client.database_names()

['admin', 'dog', 'local', 'new_cool_db', 'pubmed_ad']

In [61]:
client.pubmed_ad.alz.count()

130672

In [57]:
# used this line when I was testing with smaller batches. 
#client.pubmed_ad.alz.drop()

In [62]:
for i in client.pubmed_ad.alz.find():
    print(i)
    print()
    break

{'_id': ObjectId('598d77ccb9078c455b73ca07'), 'PMID': '28797122', 'OWN': 'NLM', 'STAT': 'Publisher', 'DA': '20170810', 'LR': '20170810', 'IS': '1553-7374 (Electronic) 1553-7366 (Linking)', 'VI': '13', 'IP': '8', 'DP': '2017 Aug 10', 'TI': 'Cross-seeding of prions by aggregated alpha-synuclein leads to transmissible spongiform encephalopathy.', 'PG': 'e1006563', 'LID': '10.1371/journal.ppat.1006563 [doi]', 'AB': "Aggregation of misfolded proteins or peptides is a common feature of neurodegenerative diseases including Alzheimer's, Parkinson's, Huntington's, prion and other diseases. Recent years have witnessed a growing number of reports of overlap in neuropathological features that were once thought to be unique to only one neurodegenerative disorder. However, the origin for the overlap remains unclear. One possibility is that diseases with mixed brain pathologies might arise from cross-seeding of one amyloidogenic protein by aggregated states of unrelated proteins. In the current study

In [63]:
# exit out of Bio
handle.close()
# close out of aws ssh tunnel
server.close()

NameError: name 'handle' is not defined